# Lab05: TF-IDF.

- MSSV: LÊ ĐOÀN CÔNG ẢNH
- Họ và tên: 1712282

## Yêu cầu bài tập

**Cách làm bài**


Bạn sẽ làm trực tiếp trên file notebook này; từ `TODO` cho biết những phần mà bạn cần phải làm.

Bạn có thể thảo luận ý tưởng cũng như tham khảo các tài liệu, nhưng *code và bài làm phải là của bạn*. 

Nếu vi phạm thì sẽ bị 0 điểm cho bài tập này.

**Cách nộp bài**

Trước khi nộp bài, rerun lại notebook (`Kernel` -> `Restart & Run All`).

Sau đó, tạo thư mục có tên `MSSV` của bạn (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) Chép file notebook, file `tf_idf_data.txt` của các bạn (nếu file này kích thước lớn các bạn có thể chép link vào `link_data.txt`), nén thư mục `MSSV` này lại và nộp trên moodle.

**Nội dung bài tập**

Cài đặt một web crawler để thu thập và biểu diễn dữ liệu bằng không gian vector và trọng số TF-IDF: https://en.wikipedia.org/wiki/Web_mining.

## Nội dung bài tập

## 1. Không gian vector

- Một vector 2 chiều có thể được viết dưới dạng `[x,y]`
- Một vector 3 chiều có thể được viết dưới dạng `[x,y,z]`
![vector](vector.png)

- Đặt $V$ là tập hợp các đặc trưng trong thể hiện dữ liệu.
- Bất kỳ mẫu dữ liệu nào đều có thể được biểu diễn dưới dạng một vectơ với $\vert V\vert$ chiều
- Ví dụ: giả sử chúng ta có 3 đặc trưng là các từ dog, bite, man. Giá trị 1 thể hiện từ đó xuất hiện 1 lần, 0 là không xuất hiện.  
![space](space.png)


## 2. TF-IDF (Term Frequency - Inverse Document Frequency)
TF-IDF cho biết độ quan trọng của một từ đối với một tài liệu trong ngữ liệu.

TF- Term Frequency : dùng để ước lượng tần xuất xuất hiện của từ trong văn bản. Tuy nhiên với mỗi văn bản thì có độ dài khác nhau số lần xuất hiện của từ có thể nhiều hơn . Vì vậy số lần xuất hiện của từ sẽ được chuẩn hóa bằng cách chia cho độ dài của văn bản (tổng số từ trong văn bản đó).
    
$$tf_{t}=\dfrac{f(t,d)}{\sum_{i \in d}f(i,d)}$$ 

- $f(t,d)$ là số lần xuất hiện từ $t$ trong văn bản $d$.

IDF- Inverse Document Frequency: Khi tính tần số xuất hiện TF thì các từ đều được coi là quan trọng như nhau. Tuy nhiên có một số từ thường được được sử dụng nhiều nhưng không quan trọng để thể hiện ý nghĩa của đoạn văn như "is", "the"... (các từ chức năng). Ta cần giảm độ quan trọng của những từ này.

$$idf_t=\log \left(\dfrac{n}{df_t}\right)$$

- *n* là số văn bản.

- $df_t$ là số văn bản xuất hiện từ t

**TF-IDF:** $$tf_{t} \times idf_t$$

- $tf_{t}$ càng lớn tần suất xuất hiện của từ trong văn bản càng lớn.
- $idf_t$ càng lớn từ hiếm khi xuất hiện trong tập dữ liệu.
- **Giả định** những đặc trưng quan trọng nhất là những đặc trưng hiếm xuất hiện.

## 3. Thu thập và thể hiện dữ liệu
Thu thập dữ liệu bắt đầu từ URL: https://en.wikipedia.org/wiki/Web_mining.
- Gọi $D$ là một tập văn bản chứa *n* văn bản: $D=\left\{d_1,d_2,...,d_n\right\}$.
- $V=\left\{v_1,v_2,...v_{\vert V \vert}\right\}$ là từ điển (tất cả các từ phân biệt trong dữ liệu thu được). $\vert V \vert$ là kích thước từ điển.
- Một trọng số $w_{ij}$ được gán cho từ $t_i$ của văn bản dj thuộc $D$ xác định mức quan trọng của $t_i$ trong văn bản $d_j$. Từ không xuất hiện trong $d_j$ có $w_{ij}=0$.
- Mỗi văn bản $d_j$ được thể hiện dưới dạng vector $\mathbf{d_j}= [w_{1j},w_{2j},...,w_{\vert V \vert j}]$
- Thể hiện dữ liệu bằng một ma trận M kích thước $n \times \vert V \vert$ => mỗi hàng thể hiện một văn bản.

In [1]:
import requests
import re
from bs4 import BeautifulSoup
from bs4.element import Comment
import string
import pickle

def get_urls(url):
    try:
        r = requests.get(url)
    except requests.exceptions.ConnectionError as e:    # This is the correct syntax
        print(e)
        return None
    # r = requests.get(url)
    # TODO
    # Lấy các url nằm trong trang web của url này, lưu lại vào biến urls
    regex = "(?:(?:https?|ftp):\/\/)[\w/\-?=%.]+\.[\w/\-&?=%.]+"
    data = r.text
    urls = re.findall(regex, data)

    # remove duplicates
    urls = list(set(urls))
    return urls


def get_urls_recursive(start_url, limit):
    urls = [start_url]
    for url in urls:
        # TODO
        # Lấy các url nằm trong trang web của url này, lưu lại vào biến new_urls
        # Với mỗi url mới trong new_urls:
        #   Nếu nó chưa nằm trong urls thì thêm nó vô  
        # Nếu kích thước của urls vượt quá limit thì dừng và xóa phần dư thừa
        new_urls = get_urls(url)
        for i in new_urls:
            if not i in urls:
                urls.append(i)
        
        if (len(urls) >= limit):
            urls = urls[0:limit]
            break
    return urls
url_list = get_urls_recursive('https://en.wikipedia.org/wiki/Web_mining', 200)

def text_filter(element):
    # TODO
    # Cài đặt lại như Lab02
    if element.parent.name in ['style', 'title', 'script', 'head', '[document]', 'class', 'a', 'li']:
        return False
    elif isinstance(element, Comment):
        '''Opinion mining?'''
        return False
    elif re.match(r"[\s\r\n]+",str(element)): 
        '''space, return, endline'''
        return False
    return True

def wordList(url):
    # TODO
    # Cài đặt lại như Lab02
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")
        text = soup.findAll(text=True)
        filtered_text = list(filter(text_filter, text)) # list của các chuỗi
        word_list = []
    except:    # This is the correct syntax
        print("Error!")
        return None

    # TODO:
    # Với mỗi chuỗi trong filtered_text:
    #   Thay thế các dấu câu thành khoảng trắng (gợi ý: danh sách các dấu câu: string.punctuation; thay thế: .replace(...))
    #   Tách chuỗi bởi khoảng trắng (.split(...))
    #   Thêm các từ vừa được tách ra vào word_list
    
    # remove all punctuation in filered_text
    for i in range(len(filtered_text)):
        for ele in filtered_text[i]:
            if ele in string.punctuation:
                filtered_text[i] = filtered_text[i].replace(ele, " ")

            # remove '\n' from filtered_text
            filtered_text[i] = filtered_text[i].replace('\n', " ")
            filtered_text[i] = filtered_text[i].replace('–', " ")
            filtered_text[i] = filtered_text[i].replace('\xa0', " ")
    
    # split by ' ' from filtered_text
    for i in filtered_text:
        word_list += i.split(' ')

    # remove all element '' in word_list
    word_list = list(filter(lambda a: a != '', word_list))
    return word_list

def read_url(url, url_idx, data):
    # TODO
    # Cài đặt lại như Lab02
    word_list = wordList(url)
    # TODO
    # Với mỗi từ w trong word_list:
    #   Nếu w chưa có trong data thì khởi tạo data[w] = [[url_idx], 1]
    #   Ngược lại thì thêm url_idx vào data[w][0] (nếu chưa có) và tăng data[w][1] lên 1 đơn vị
    
    if (word_list == None):
        return
    for w in word_list:
        if w in data:
            # data[w][1] += 1
            for i in range(0, len(data[w][0])):
                    if (data[w][0][i] == url_idx):
                        data[w][1][i] = data[w][1][i] + 1

            if not url_idx in data[w][0]:
                data[w][0].append(url_idx)
                data[w][1].append(1)
        else:
            data[w] = [[url_idx], [1]]

data = {}
for url_index, url in enumerate(url_list, 1):
    print('Executing is '+ str(url_index))
    read_url(url, url_index, data)

data

Executing is 1
Executing is 2
Executing is 3
Executing is 4
Executing is 5
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Executing is 6
Executing is 7
Executing is 8
Executing is 9
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Executing is 10
Executing is 11
Executing is 12
Executing is 13
Executing is 14
Executing is 15
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Executing is 16
Executing is 17
Executing is 18
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Executing is 19
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Executing is 20
Executing is 21
Executing is 22
Executing is 23
Executing is 24
Executing is 25
Executing is 26
Executing is 27
Executing is 28
Error!
Executing is 29
Executing is 30
Executing is 31
Executing is 32
Executing is 33
Some characters could not be decoded, and were replace

   83,
   84,
   87,
   88,
   90,
   96,
   98,
   100,
   101,
   104,
   105,
   106,
   107,
   110,
   112,
   113,
   114,
   118,
   120,
   121,
   122,
   124,
   129,
   131,
   134,
   137,
   138,
   139,
   140,
   142,
   143,
   144,
   145,
   146,
   148,
   149,
   150,
   152,
   154,
   155,
   156,
   160,
   162,
   163,
   165,
   167,
   168,
   169,
   172,
   176,
   191],
  [8,
   1,
   1,
   1,
   7,
   8,
   8,
   8,
   8,
   8,
   8,
   8,
   8,
   8,
   8,
   8,
   8,
   8,
   3,
   8,
   7,
   8,
   8,
   7,
   6,
   8,
   8,
   8,
   8,
   7,
   7,
   6,
   11,
   8,
   2,
   3,
   8,
   5,
   7,
   8,
   7,
   8,
   3,
   8,
   8,
   3,
   8,
   3,
   8,
   8,
   5,
   7,
   8,
   1,
   8,
   8,
   8,
   8,
   3,
   8,
   2,
   8,
   8,
   8,
   8,
   8,
   8,
   8,
   5,
   8,
   8,
   6,
   8,
   1,
   2,
   1]],
 'variety': [[2,
   27,
   50,
   54,
   56,
   57,
   58,
   59,
   63,
   64,
   65,
   66,
   67,
   68,
   70,
   71,
   72,
   73,
   

In [2]:
numberOfWordInUrls = {}
for index in range(1, len(url_list) + 1):
    for word in data:
        if (index in data[word][0]):
            if numberOfWordInUrls.get(index) == None:
                numberOfWordInUrls[index] = 1
            else:
                numberOfWordInUrls[index] = numberOfWordInUrls[index] + 1
numberOfWordInUrls

{1: 474,
 2: 1146,
 4: 88,
 5: 8695,
 6: 101,
 7: 34,
 8: 439,
 9: 15219,
 10: 281,
 11: 1362,
 12: 622,
 13: 90,
 14: 292,
 15: 28992,
 16: 22,
 17: 31,
 18: 34773,
 19: 46269,
 20: 320,
 21: 35,
 22: 16,
 23: 257,
 24: 1075,
 26: 474,
 27: 1511,
 29: 245,
 30: 160,
 31: 279,
 32: 69,
 33: 33692,
 34: 292,
 35: 4991,
 36: 126,
 37: 8,
 38: 763,
 39: 257,
 40: 24,
 41: 5642,
 42: 233,
 43: 69,
 44: 98,
 45: 11607,
 47: 370,
 48: 444,
 49: 30,
 50: 477,
 51: 7,
 52: 24,
 53: 322,
 54: 1555,
 55: 251,
 56: 1161,
 57: 1201,
 58: 1150,
 59: 1146,
 60: 35,
 61: 1452,
 62: 265,
 63: 1157,
 64: 1153,
 65: 1124,
 66: 1196,
 67: 1131,
 68: 1167,
 69: 71,
 70: 1188,
 71: 1145,
 72: 1366,
 73: 1166,
 74: 858,
 75: 374,
 76: 1212,
 77: 334,
 78: 1129,
 79: 1208,
 80: 1517,
 81: 1437,
 82: 58,
 83: 1210,
 84: 1205,
 85: 1770,
 86: 1680,
 87: 1230,
 88: 1171,
 89: 34,
 90: 1480,
 91: 1426,
 92: 1354,
 93: 190,
 94: 301,
 95: 35,
 96: 1474,
 97: 1675,
 98: 1609,
 99: 1408,
 100: 1365,
 101: 1176,
 10

In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')
english_stopwords


[nltk_data] Downloading package stopwords to /Users/macos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [4]:
#TODO
import numpy as np
import math

n_urls=200
n_words = len(data)
M=np.zeros(shape=(n_urls,n_words),dtype=float)

for idx_matrix in range(0, n_urls):
    column = 0
    idx_url = idx_matrix + 1
    for word in data:
        if not idx_url in data[word][0]:
            M[idx_matrix][column] = 0
        else:
            index_word_data = data[word][0].index(idx_url)
            tf = data[word][1][index_word_data] / numberOfWordInUrls[idx_url]
            idf = 1
            if word in english_stopwords:
                idf = math.log(n_urls / len(data[word][0]))
            tfIdf = tf * idf
            M[idx_matrix][column] = tfIdf
        column = column + 1
M

array([[0.08227848, 0.08227848, 0.0021097 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.004363  , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00181818, ..., 0.00181818, 0.00181818,
        0.00181818]])

In [5]:
#TODO save data to txt file using numpy
with open("tf_idf_data.txt", "w",encoding="utf-8") as f:
    np.savetxt(f, M)
print('Done')

Done
